## 시작
패키지 install, 함수 정의

https://wikidocs.net/192931


In [1]:
pip install jpype1

In [2]:
pip install konlpy

In [3]:
pip install sentencepiece

In [4]:
#한글파일 unzip시 깨짐 encoding decoding으로 방지하는 함수, num개만 압축해제
def unzip(source_path, dest_path):
  with zipfile.ZipFile(source_path, 'r') as zf:
    zipInfo = zf.infolist()
    for member in zipInfo:
      try:
        #print(member.filename.encode('cp437').decode('euc-kr','ignore'))
        member.filename = member.filename.encode('cp437').decode('euc-kr','ignore')
        zf.extract(member,dest_path)
      except:
        print(source_path)
        raise Exception('??')

## 데이터 준비
데이터 다운로드

In [5]:
from google.colab import drive
import shutil
import zipfile
import os

import json

In [6]:
drive.mount('/content/drive')

source_path = '/content/drive/MyDrive/ML_Data/146.낚시성 기사 탐지 데이터.zip'
destination_path = '/content/146.낚시성 기사 탐지 데이터.zip'
shutil.copyfile(source_path,destination_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/146.낚시성 기사 탐지 데이터.zip'

## Data 다운로드/압축해제
원문 Clickbait Direct 만 뽑아 새로운 데이터셋 생성 - 본문/제목 다중입력

In [7]:
#데이터 저장을 위한 경로 만들기
root_path = '/content/root'
training_data_path = os.path.join(root_path,'Training')
test_data_path = os.path.join(root_path,'Test')

p = [root_path,training_data_path,test_data_path]
for i in p:
  if not os.path.exists(i):
    os.mkdir(i)

In [8]:
#전체 중 Clickbait_Direct 아닌거 삭제 - Clickbait_Direct 만 processPattern 가짐
unzip('/content/146.낚시성 기사 탐지 데이터.zip','/content')
raw_data_path = '/content/146.낚시성 기사 탐지 데이터'

for root,dirs,files in os.walk(raw_data_path):
  for fn in files:
    if 'Clickbait_Direct' not in fn:
      os.remove(root+'/'+fn)



In [9]:
#(임시) zip 하나만 풀어서 dataset 만들기
a='/content/146.낚시성 기사 탐지 데이터/01.데이터/Training/02.라벨링데이터/TL_Part1_Clickbait_Direct_EC.zip'
b='/content/146.낚시성 기사 탐지 데이터/01.데이터/Training/02.라벨링데이터/TL_Part1_Clickbait_Direct_IS.zip'
unzip(a,training_data_path)
unzip(b,test_data_path)

## 데이터 전처리

본문->토큰화->벡터화(원-핫 인코딩)

1.   train_data
[본문.........],[본문..........]....
2.   token_train_data
[시퀀스시퀀스시퀀스],[시퀀스시퀀스시퀀스],[시퀀스시퀀스시퀀스]....
3. x_train
[0,0,0,0,1,0,0,,,,],[0,1,0,0,0,0,,,,,,],,,,,



데이터준비

In [60]:
import re
import numpy as np

#training_data 경로의 모든 파일 읽어오기 (json)
training_files = []
for root, dirs, files in os.walk(training_data_path):
  for fn in files:
    training_files.append(root+'/'+fn)

#각 json 파일의 본문을 리스트에 추가
train_data = np.array([])
train_labels = []

for i in training_files[:10]:
  with open(i,'r',encoding='utf-8') as file:
    data = json.load(file)

    newsContent = data.get('sourceDataInfo').get('newsContent')
    #newsContent = re.sub(r'\n',' ', newsContent) # 엔터키를 없애버리면 sentencepiece 패키지가 멍청해서 인식을 못함
    newsContent = re.sub('[()]',' ',newsContent)
    newsContent = re.sub(r'\\',' ',newsContent)
    train_data = np.append(newsContent,train_data)

    train_labels.append(int(data.get('sourceDataInfo').get('processPattern')))

#-------------------------------------------------------------------------
#위에랑 똑같은거 test에도
test_files = []
for root,dirs,files in os.walk(test_data_path):
  for fn in files:
    test_files.append(root+'/'+fn)

test_data = np.array([])
test_labels = []

for i in test_files[:10]:
  with open(i,'r',encoding = 'utf-8') as file:
    data = json.load(file)

    newsContent = data.get('sourceDataInfo').get('newsContent')
    newsContent = re.sub('[()]',' ',newsContent)
    newsContent = re.sub(r'\\',' ',newsContent)
    test_data = np.append(newsContent,test_data)

    test_labels.append(int(data.get('sourceDataInfo').get('processPattern')))

본문 - 토큰화

In [ ]:
#한국어 토큰화 위한 과정
#https://velog.io/@lighthouse97/%ED%85%8D%EC%8A%A4%ED%8A%B8-%EC%A0%84%EC%B2%98%EB%A6%AC-%ED%86%A0%ED%81%B0%ED%99%94

#토큰화 - 1 형태소 분석
from konlpy.tag import Okt

okt = Okt()
text = '아이고 배고파 ㅠㅠ 이럴수가. 컵라면을 사먹어요'

morphs = okt.morphs(text)
ps = okt.pos(text)
noun = okt.nouns(text)
print(f"형태소 분석 : {morphs}")
print(f'품사 태깅 : {ps}')
print(f'명사 추출 : {noun}')

In [62]:
#토큰화 - 2 Subword Tokenizer
#BPE 알고리즘을 구현한 구글의 Sentencepiece를 사용해 내부 단어 분리 진행

#이름.model 이 sentencepiece에서 사용되는 단어집합
#이름.vocal 이 단어 집합을 텍스트 파일로 저장한것
#모든 텍스트파일을 하나의 txt에 쳐넣고 학습해 model을 만들고 각각의 train_data 안에 있는 데이터에 sp.encode를 해주면 된다.

import sentencepiece as spm
#모든 텍스트 하나의 txt에 쳐넣어 sentencepiece 학습시키기
with open('/content/test1.txt','w',encoding='utf-8') as f:
  for i in range(0,len(train_data)):
    f.write(train_data[i])
  for i in range(0,len(test_data)):
    f.write(test_data[i])

spm.SentencePieceTrainer.Train('--input=test1.txt --model_prefix=mine --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

#Sentence piece 객체 생성
sp = spm.SentencePieceProcessor()



In [63]:
#모델 불러와 인코딩, 디코딩 시험
sp.load('mine.model')
print(sp.encode(train_data[0], out_type=int))
print(sp.DecodeIds([3864, 2255, 4515, 392, 556, 1254, 2030, 1084, 1415, 4282, 1912, 2764, 1925, 392, 556, 916, 3247, 2780, 100, 1237, 4282, 2901, 4088, 51, 2394, 3512, 4349, 4284, 288, 43, 189, 4328, 1077, 3482, 392, 1983, 1255]))


[3864, 2255, 4515, 392, 556, 1254, 2030, 1084, 1415, 4282, 1912, 2764, 1925, 392, 556, 916, 3247, 2780, 100, 1237, 4282, 2901, 4088, 51, 2394, 3512, 4349, 4284, 288, 43, 189, 4328, 1077, 3482, 392, 1983, 1255, 1198, 98, 1742, 2019, 1177, 4003, 2417, 100, 4279, 4091, 4282, 1541, 451, 1192, 392, 556, 916, 1255, 514, 2045, 259, 615, 3694, 1983, 1198, 818, 451, 2019, 1177, 3604, 3855, 4282, 3695, 556, 292, 430, 1901, 433, 1208, 4282, 1192, 1912, 1913, 2894, 4287, 1594, 2776, 278, 392, 556, 1254, 1176, 2100, 1177, 934, 471, 1742, 3583, 3379, 1081, 1356, 1901, 1198, 3268, 100, 810, 157, 4282, 2989, 290, 4341, 2045, 3291, 954, 4302, 916, 4257, 100, 2177, 612, 4282, 846, 759, 2312, 1422, 290, 346, 2496, 1201, 451, 916, 2603, 2099, 803, 656, 11, 421, 1569, 3290, 2862, 3591, 1255, 433, 1391, 1208, 4282, 44, 2822, 2850, 151, 114, 3718, 2186, 1756, 1130, 108, 556, 1254, 3258, 4295, 2030, 3951, 22, 1530, 4472, 71, 114, 4280, 515, 3686, 4036, 3588, 1966, 1707, 4472, 4292, 150, 4282]
한동안 치솟던 아파트 매매가와

In [64]:
#train_data, test_data 전부 Tokenize

token_train_data = []
for i in range(0,len(train_data)):
  token_train_data.append(sp.encode(train_data[i], out_type = int))
token_test_data = []
for i in range(0,len(test_data))  :
  token_test_data.append(sp.encode(test_data[i],out_type=int))

본문 - 벡터변환 (원-핫 인코딩)

In [65]:
#데이터 벡터변환 (원-핫 인코딩 방식 벡터변환으로 각 기사에 어떤 단어가 사용되었는지에 대한 정보를 담고 있다.)
import numpy as np

#행 : 각 기사들, 열 : 단어 시퀀스, 값 : 기사의 해당 단어 사용 여부
def vectorize_squences(sequences, dimension=5000):
  results = np.zeros((len(sequences),dimension))
  print(results.shape)

  #i번째 원소의 기사 sequence를 np.zero 각각의 줄에 그냥 다 넣기
  for i, sequence in enumerate(sequences):
    results[i,sequence] = 1
  return results

x_train = vectorize_squences(token_train_data)
x_test = vectorize_squences(token_test_data)

(10, 5000)
(10, 5000)


라벨->벡터변환(원 핫 인코딩)


1.   train_labels
[3,4,3,6,1,2,6,3,7,3....]
2.   one_hot_train_labels
[0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0]


라벨 값
*   0-5은 processPattern 11-16
*   6-9은 processPattern 21-24


In [66]:
#라벨 원-핫 인코딩 벡터변환
def to_one_hot(labels, dimension=10):
  results = np.zeros((len(labels),dimension))
  for i, label in enumerate(labels):
    if label <= 16:
      results[i,label-11] = 1
    elif label >=21:
      results[i,label-15] = 1
    else:
      pass
  return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

In [67]:
print(one_hot_train_labels)
print(one_hot_test_labels)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


## 3.모델 만들기


입력 Layer 2개- 본문/제목
은닉 Layer
출력 Layer 10개


In [68]:
#시퀀스-벡터 한 데이터 1:9로 나누기
x_val = x_train[:5]
partial_x_train = x_train[5:]

#label one_hot_encoding 한 데이터 1:9로 나누기
y_val = one_hot_train_labels[:5]
partial_y_train = one_hot_train_labels[5:]

## 4. 모델 구성
출력 클래스 개수 10개

In [71]:
from keras import models
from keras import layers

#Sequential 모델 정의
model = models.Sequential()
#입력층 - 64개의 뉴런, relu 활성화 함수, 10000개의 입력데이터
model.add(layers.Dense(64, activation='relu',input_shape=(5000,)))
#은닉층
model.add(layers.Dense(64, activation='relu'))
#출력층
model.add(layers.Dense(10, activation='softmax'))


model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
#keras Sequential model에 나눈 데이터 중 9쪽 데이터를 넣어 train하고 validation으로는 1쪽을 넣음
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=1,
                    validation_data=(x_val,y_val))

Epoch 1/20
5/5 [==============================] - 1s 62ms/step - loss: 2.1662 - accuracy: 0.2000 - val_loss: 1.9411 - val_accuracy: 0.2000
Epoch 2/20
5/5 [==============================] - 0s 14ms/step - loss: 1.0327 - accuracy: 1.0000 - val_loss: 1.6332 - val_accuracy: 0.6000
Epoch 3/20
5/5 [==============================] - 0s 13ms/step - loss: 0.4098 - accuracy: 1.0000 - val_loss: 1.5068 - val_accuracy: 0.6000
Epoch 4/20
5/5 [==============================] - 0s 16ms/step - loss: 0.1739 - accuracy: 1.0000 - val_loss: 1.3999 - val_accuracy: 0.6000
Epoch 5/20
5/5 [==============================] - 0s 14ms/step - loss: 0.0900 - accuracy: 1.0000 - val_loss: 1.3589 - val_accuracy: 0.6000
Epoch 6/20
5/5 [==============================] - 0s 16ms/step - loss: 0.0514 - accuracy: 1.0000 - val_loss: 1.3129 - val_accuracy: 0.8000
Epoch 7/20
5/5 [==============================] - 0s 19ms/step - loss: 0.0322 - accuracy: 1.0000 - val_loss: 1.2950 - val_accuracy: 0.8000
Epoch 8/20
5/5 [===========

In [74]:
pred = model.predict(x_test)
np.argmax(pred[0])

1/1 [==============================] - 0s 22ms/step


0

새로운 데이터로 테스트해보기
본문->토큰화->벡터화->모델 분석 ->argmax 함수

In [81]:
text = '''강도형 해양수산부 장관 후보자가 후쿠시마 오염수 방류와 관련해 수산물 안전을 확보하는데 최선을 다하겠다는 뜻을 밝혔다.

7일 강도형 후보자는 지명 이후 처음 서울 마포구에 마련된 인사 청문 준비 사무실로 출근했다.

강 후보자는 수산물 안전 우려를 묻는 취재진의 질문에 "국민에게 위해가 되는 일이 없게 하겠다"며 "국내외의 검사 지점을 활용해 방사능 검사를 계속하겠다"고 말했다.

또 "항만 분야는 배후 부지를 활용해 경제적 가치를 가질 수 있도록 하고 그린쉽 친환경선박을 잘 챙길 것"이라고 했다.

강 후보자는 "해양수산 관련 산업구조를 첨단화해 국제사회 선도 국가로 갈 수 있는 전략을 짜야 한다면서 "소통하면서 타 부처와 협업할 것"이라고 말했다.

정부의 R&D 예산 삭감에 대해서는 "이번 연구개발 개편은 비효율적인 것을 걷어내는 작업이었다고 생각한다"며 "부족한 것이 있다면 2025년, 2026년 예산에 넣어 보강하도록 노력하겠다"고 덧붙였다.

강 후보자는 해양과학 전문 연구원 출신으로 지난 2월부터 한국해양과학기술원(KIOST) 원장을 맡아왔다.'''

token_text = sp.encode(text,out_type=int)
vec_text = vectorize_squences(token_text)
pred1 = model.predict(vec_text)

[3864, 2255, 4515, 392, 556, 1254, 2030, 1084, 1415, 4282, 1912, 2764, 1925, 392, 556, 916, 3247, 2780, 100, 1237, 4282, 2901, 4088, 51, 2394, 3512, 4349, 4284, 288, 43, 189, 4328, 1077, 3482, 392, 1983, 1255, 1198, 98, 1742, 2019, 1177, 4003, 2417, 100, 4279, 4091, 4282, 1541, 451, 1192, 392, 556, 916, 1255, 514, 2045, 259, 615, 3694, 1983, 1198, 818, 451, 2019, 1177, 3604, 3855, 4282, 3695, 556, 292, 430, 1901, 433, 1208, 4282, 1192, 1912, 1913, 2894, 4287, 1594, 2776, 278, 392, 556, 1254, 1176, 2100, 1177, 934, 471, 1742, 3583, 3379, 1081, 1356, 1901, 1198, 3268, 100, 810, 157, 4282, 2989, 290, 4341, 2045, 3291, 954, 4302, 916, 4257, 100, 2177, 612, 4282, 846, 759, 2312, 1422, 290, 346, 2496, 1201, 451, 916, 2603, 2099, 803, 656, 11, 421, 1569, 3290, 2862, 3591, 1255, 433, 1391, 1208, 4282, 44, 2822, 2850, 151, 114, 3718, 2186, 1756, 1130, 108, 556, 1254, 3258, 4295, 2030, 3951, 22, 1530, 4472, 71, 114, 4280, 515, 3686, 4036, 3588, 1966, 1707, 4472, 4292, 150, 4282]
(316, 5000)
10/1

In [83]:
np.argmax(pred1[0])

0